In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange


In [40]:
bs = 8
sl = 128
d = 256
x = torch.randn(bs, sl, d).cuda()
context = torch.randn(bs, sl-16, d).cuda()

In [41]:
to_qkv = nn.Linear(d, d*3).cuda()
q, k, v = to_qkv(x).chunk(3, -1)

In [42]:
h = 8
scale = (d//h)**-0.5
q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), (q, k, v))

In [43]:
# q_, k_, v_ = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), (q, k, v))
# q_, k_, v_ = map(lambda t: rearrange(t, 'b h n d -> b n (h d)', h = h), (q_, k_, v_))
# assert (q == q_).all()
# assert (k == k_).all()
# assert (v == v_).all()

In [44]:
dots1 = torch.einsum('bhid,bhjd->bhij', q*scale, k)
# dots1 = dots1*scale

In [45]:
q, k, v = map(lambda t: rearrange(t, 'b h n d -> (b h) n d', h = h), (q, k, v))
q.shape

torch.Size([64, 128, 32])

In [46]:
dots2 = torch.bmm(q*scale, k.transpose(-2,-1))

In [47]:
dots1.shape

torch.Size([8, 8, 128, 128])

In [48]:
dots2.shape

torch.Size([64, 128, 128])

In [49]:
dots1_ = dots1.contiguous().view(-1, 128, 128)

In [50]:
(dots1_ == dots2).all()

tensor(True, device='cuda:0')

In [51]:
(dots1_ - dots2)[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<SelectBackward>)